In [1]:
import tensorflow as tf

from tensorflow.examples.tutorials.mnist import mnist, input_data
data_sets = input_data.read_data_sets("/tmp/data/", one_hot=True)


/home/foriequal0/.local/share/virtualenvs/tf-tutorial-pgL6-UTG/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6
  return f(*args, **kwds)


Extracting /tmp/data/train-images-idx3-ubyte.gz
Extracting /tmp/data/train-labels-idx1-ubyte.gz
Extracting /tmp/data/t10k-images-idx3-ubyte.gz
Extracting /tmp/data/t10k-labels-idx1-ubyte.gz


In [2]:
learning_rate = 0.01
training_epochs = 25
batch_size = 100
display_step = 1
logs_path = '/tmp/tensorflow_logs/tensorboard-adv'

n_hidden_1 = 256
n_hidden_2 = 256
n_input = mnist.IMAGE_PIXELS
n_classes = mnist.NUM_CLASSES

x = tf.placeholder(tf.float32, [None, n_input], name='Input')
y = tf.placeholder(tf.float32, [None, n_classes], name='Label')

In [3]:
def multilayer_perceptron(x, weights, biases):
    layer_1 = tf.add(tf.matmul(x, weights['w1']), biases['b1'])
    layer_1 = tf.nn.relu(layer_1)
    
    tf.summary.histogram("relu1", layer_1)
    
    layer_2 = tf.add(tf.matmul(layer_1, weights['w2']), biases['b2'])
    layer_2 = tf.nn.relu(layer_2)
    
    tf.summary.histogram("relu2", layer_2)
    
    out_layer = tf.add(tf.matmul(layer_2, weights['w3']), biases['b3'])
    return out_layer

weights = {
    'w1': tf.Variable(tf.random_normal([n_input, n_hidden_1]), name='W1'),
    'w2': tf.Variable(tf.random_normal([n_hidden_1, n_hidden_2]), name='W2'),
    'w3': tf.Variable(tf.random_normal([n_hidden_2, n_classes]), name='W3')
}
biases = {
    'b1': tf.Variable(tf.random_normal([n_hidden_1]), name='b1'),
    'b2': tf.Variable(tf.random_normal([n_hidden_2]), name='b2'),
    'b3': tf.Variable(tf.random_normal([n_classes]), name='b3')
}

In [4]:
with tf.name_scope('Model'):
    pred = multilayer_perceptron(x, weights, biases)
    
with tf.name_scope('Loss'):
    loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=pred, labels=y))

with tf.name_scope('SGD'):
    optimizer = tf.train.GradientDescentOptimizer(learning_rate)

    grads = tf.gradients(loss, tf.trainable_variables())
    grads = list(zip(grads, tf.trainable_variables()))
    
    apply_grads = optimizer.apply_gradients(grads_and_vars=grads)
    
with tf.name_scope('Accuracy'):
    acc = tf.equal(tf.argmax(pred, 1), tf.argmax(y, 1))
    acc = tf.reduce_mean(tf.cast(acc, tf.float32))

In [5]:
init = tf.global_variables_initializer()

tf.summary.scalar("loss", loss)
tf.summary.scalar("accuracy", acc)
for var in tf.trainable_variables():
    tf.summary.histogram(var.name, var)
for grad, var in grads:
    tf.summary.histogram(var.name + '/gradient', grad)

merged_summary_op = tf.summary.merge_all()

INFO:tensorflow:Summary name W1:0 is illegal; using W1_0 instead.
INFO:tensorflow:Summary name W2:0 is illegal; using W2_0 instead.
INFO:tensorflow:Summary name W3:0 is illegal; using W3_0 instead.
INFO:tensorflow:Summary name b1:0 is illegal; using b1_0 instead.
INFO:tensorflow:Summary name b2:0 is illegal; using b2_0 instead.
INFO:tensorflow:Summary name b3:0 is illegal; using b3_0 instead.
INFO:tensorflow:Summary name W1:0/gradient is illegal; using W1_0/gradient instead.
INFO:tensorflow:Summary name W2:0/gradient is illegal; using W2_0/gradient instead.
INFO:tensorflow:Summary name W3:0/gradient is illegal; using W3_0/gradient instead.
INFO:tensorflow:Summary name b1:0/gradient is illegal; using b1_0/gradient instead.
INFO:tensorflow:Summary name b2:0/gradient is illegal; using b2_0/gradient instead.
INFO:tensorflow:Summary name b3:0/gradient is illegal; using b3_0/gradient instead.


In [6]:
with tf.Session() as sess:
    sess.run(init)

    summary_writer = tf.summary.FileWriter(logs_path,
                                            graph=tf.get_default_graph())

    for epoch in range(training_epochs):
        avg_cost = 0.
        total_batch = int(data_sets.train.num_examples/batch_size)

        for i in range(total_batch):
            batch_xs, batch_ys = data_sets.train.next_batch(batch_size)
            # Run optimization op (backprop), cost op (to get loss value)
            # and summary nodes
            _, c, summary = sess.run([apply_grads, loss, merged_summary_op],
                                     feed_dict={x: batch_xs, y: batch_ys})
            
            summary_writer.add_summary(summary, epoch * total_batch + i)
            
            avg_cost += c / total_batch

        if (epoch+1) % display_step == 0:
            print("Epoch: {}, cost={:.9f}".format(epoch+1, avg_cost))

    print("Optimization Finished!")

    # Test model
    # Calculate accuracy
    print("Accuracy:", acc.eval({x: data_sets.test.images, y: data_sets.test.labels}))

    print("Run the command line:\n"
          "--> tensorboard --logdir=/tmp/tensorflow_logs ")

Epoch: 1, cost=57.859624217
Epoch: 2, cost=12.999777106
Epoch: 3, cost=7.969581386
Epoch: 4, cost=5.558275228
Epoch: 5, cost=4.205380959
Epoch: 6, cost=3.297019291
Epoch: 7, cost=2.648041704
Epoch: 8, cost=2.154125501
Epoch: 9, cost=1.823525820
Epoch: 10, cost=1.544961842
Epoch: 11, cost=1.328512219
Epoch: 12, cost=1.127693035
Epoch: 13, cost=0.986281166
Epoch: 14, cost=0.855102292
Epoch: 15, cost=0.734404508
Epoch: 16, cost=0.644446341
Epoch: 17, cost=0.556785919
Epoch: 18, cost=0.499766647
Epoch: 19, cost=0.435068668
Epoch: 20, cost=0.386770025
Epoch: 21, cost=0.344982265
Epoch: 22, cost=0.301302200
Epoch: 23, cost=0.272461303
Epoch: 24, cost=0.248432595
Epoch: 25, cost=0.221313677
Optimization Finished!
Accuracy: 0.9317
Run the command line:
--> tensorboard --logdir=/tmp/tensorflow_logs 
